<a href="https://colab.research.google.com/github/BraedenCu/ObjectDetectionWithPytorchTutorial/blob/master/vexaiPytorch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages (should be automatically installed in colab)
If you get an import error, install each of them using python3 -m pip install ____

Next we clone my fork of the pytorch vision repository, which contains scripts we will use later. Then clone some useful scripts for data manipluation from my repository. Finally, create directories for annotations and images.

In [ ]:
#EVERYTIME KERNEL DIES, YOU NEED TO REUPLOAD XMLS AND IMAGES

import torch
import torchvision
import pycocotools
import os

%cd /root/
#need to clone my fork, elsewise it will throw import error when using one of the libraries
#remove and previous installation, clone the most recent
!rm -r vision
!git clone https://github.com/BraedenCu/vision.git
!cd vision
!git checkout v0.6.0

%cd ~
#remove any previous clone of the repo and install the latest release 
!rm -r scriptsForObjectDetectionDataPreparation/
!git clone https://github.com/BraedenCu/scriptsForObjectDetectionDataPreparation.git
!mkdir annotations
!mkdir images

Here we are first removing any prevois copys of the following scripts and updating them. These scripts will be used later for training.


In [ ]:
#in order to gain access to functions defined in engine.py/utils.py/etc. we need to copy
#the python scripts to root
%cd /root/
#remove old versions of the python scripts and replace with updated ones
!rm -r engine.py
!rm -r coco_eval.py
!rm -r coco_utils.py
!rm -r utils.py
%cd vision
%cd references
%cd detection
#copy files to root
!cp engine.py -b /root/
!cp utils.py -b /root/
!cp coco_utils.py -b /root/
!cp coco_eval.py -b /root/
#change directory back to root
%cd /root/
!ls

In [ ]:
%cd ~/scriptsForObjectDetectionDataPreparation/
#convert xml_to_csv, if it failes to create do to empty csv, be sure to check that you have uploaded your images and annottions
#remove and reupdate
!rm -r vex_labels.csv
!python xml_to_csv.py

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import pandas as pd

#function to return data when given filename
def parse_one_annot(path_to_csv_file, filename):
   data = pd.read_csv(path_to_csv_file)
   boxes_array = data[data["filename"] == filename][["xmin", "ymin",        
   "xmax", "ymax"]].values
   #print(boxes_array)
   return boxes_array

   
class vex(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = "/root/images" #sorted(os.listdir(os.path.join(root, "images")))
        self.csv_path = "/root/scriptsForObjectDetectionDataPreparation/vex_labels.csv"

    def __getitem__(self, idx):
        # load images ad masks
        #simply just repeat 1 rather than search for 0.jpeg, which does not exist
        if idx == 0:
          idx = 1
        
        img_path = "/root/images/{}{}".format(idx, ".jpeg")
        #print(img_path)
        img = Image.open(img_path).convert("RGB")
        
        # convert the PIL Image into a numpy array
        #box_list = parse_one_annot(self.csv_path, self.imgs[idx])
        box_list = parse_one_annot(self.csv_path, img_path)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        # 5 classes, so five boxes to be either zero or one
        num_objs = 5
        labels = torch.ones((num_objs), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
    
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        #target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        #if self.transforms is not None:
        img = self.transforms(img)
            #target = self.transforms(img)
        return img, target

    def __len__(self):
        #return len(self.imgs)
        return 204

In [ ]:
#makes sure image path is correct
#img_path = "/root/images/{}{}".format(idx, ".jpeg")
#print(img_path)
#img = Image.open(img_path).convert("RGB")

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 5  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

In [ ]:
def get_model_instance_segmentation(num_classes):
   # load an object detection model pre-trained on COCO
   model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get the number of input features for the classifier
   in_features = model.roi_heads.box_predictor.cls_score.in_features
   # replace the pre-trained head with a new on
   model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
   return model


In [ ]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    #if train:
        #transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
import utils
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = vex('/root/', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

In [ ]:
%cd /root

/root


In [ ]:
from engine import train_one_epoch, evaluate
import utils

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 5
    # use our dataset and defined transformations
    dataset = vex('/root/', get_transform(train=True))
    dataset_test = vex('/root/', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    #dataset = torch.utils.data.Subset(dataset, indices[:-50])
    #dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 5

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

main()


In [ ]:
#save model to training/models
#remove prior saves
!rm -r training
!mkdir training
torch.save(model.state_dict(), "/root/training/model")

In [ ]:
from PIL import ImageDraw

loaded_model = get_model_instance_segmentation(num_classes = 5)
loaded_model.load_state_dict(torch.load("/root/training/model"))
dataset_test = vex('/root/', get_transform(train=False))
idx = 0
img, _ = dataset_test[idx]
label_boxes = np.array(dataset_test[idx][1]["boxes"])
#put the model in evaluation mode
loaded_model.eval()
with torch.no_grad():
   prediction = loaded_model([img])
image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
draw = ImageDraw.Draw(image)
# draw groundtruth
for elem in range(len(label_boxes)):
   draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
   (label_boxes[elem][2], label_boxes[elem][3])], 
   outline ="green", width =3)
for element in range(len(prediction[0]["boxes"])):
   boxes = prediction[0]["boxes"][element].cpu().numpy()
   score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                    decimals= 4)
   if score > 0.45:
      draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
      outline ="red", width =3)
      draw.text((boxes[0], boxes[1]), text = str(score))
image